<a id='1.1'></a>
## Install all the packages through FinRL library


In [ ]:
## install finrl library
!pip install git+https://github.com/amritgos/FinRL.git

  Cloning https://github.com/amritgos/FinRL.git to /tmp/pip-req-build-kwfqdo_5
  Running command git clone -q https://github.com/amritgos/FinRL.git /tmp/pip-req-build-kwfqdo_5


In [ ]:
!git clone -q https://github.com/amritgos/FinRL.git

In [ ]:
cd FinRL


<a id='1.2'></a>
## Packages Required 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import sys
sys.path.append("../FinRL")

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Downloading Data


In [ ]:
print(config.SENSEX_30_TICKER)

In [ ]:
df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2021-01-01',
                     ticker_list = config.SENSEX_30_TICKER).fetch_data()

In [ ]:
df.head()

In [ ]:
df.shape

# Preprocessing Data


In [ ]:
vix_df = pd.read_csv('datasets/IndiaVIX_data.csv')

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=True,
                    use_vix=True,
                    vix_df=vix_df,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

In [ ]:
df.shape

In [ ]:
df.head()

<a id='4'></a>
# Setting up Environment


## Training data split: 2009-01-01 to 2018-12-31

In [ ]:
train = data_split(df, '2009-01-01','2019-01-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)

In [ ]:
train.head()

## Environment for Portfolio Optimization


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


In [ ]:
env_kwargs = {
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "turbulence_threshold": None,
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<a id='5'></a>
# A2C Algorithm

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.001}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=60000)

## Trading
Assume that we have Rs10,00,000 initial capital at 2019-01-01. We use the A2C model to trade SENSEX stocks.

In [ ]:
data_turbulence = processed[(df.date<'2019-01-01') & (df.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,0.95)

In [ ]:
data_vix = processed[(df.date<'2019-01-01') & (df.date>='2009-01-01')]
insample_vix = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
vix_threshold = np.quantile(insample_vix.VIX.values,0.95)

In [ ]:
env_kwargs = {
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "turbulence_threshold": turbulence_threshold,
    "vix_threshold": vix_threshold,
    "reward_scaling": 1e-4
    
}

In [ ]:
trade = data_split(df,'2019-01-01', '2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [ ]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

In [ ]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Backtesting methodology

<a id='6.1'></a>
## BackTestStats


In [ ]:
from pyfolio import timeseries
s1 = get_daily_return(df_daily_return, value_col_name="account_value")
df_daily_return['daily_return'] = s1.values
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [ ]:
print("==============DRL Strategy Stats===========")
perf_stats_all

<a id='6.2'></a>
## BackTestPlot

In [ ]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^BSESN', start='2019-01-01', end='2021-01-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)